<h1>Trabalho 1 de SAD</h1>

<h3>1 - Posse de bola influencia no resultado final da partida?</h3>
<h4>Banco:</h4> 'Match' <h4>Colunas:</h4> 'home_team_goal', 'away_team_goal', 'possession'

<h3>2 - Time que menos modifica seus jogadores tem melhor colocacao na tabela?</h3>
<h4>Banco:</h4> 'Match' <h4>Colunas:</h4> 'home_player[1..11]', 'match_api_id', 'session', 'home_team_api_id'\
       'away_team_api_id', 'away_player[1..11]', 'home_team_goal', 'away_team_goal'

<h3>3 - Time que mais chuta ao gol influencia no resultado final da partida?</h3>
<h4>Banco:</h4> 'Match' <h4>Colunas:</h4> 'home_team_goal', 'away_team_goal', 'shoton', 'shotoff'

In [5]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

path_banco = os.getenv("HOME") + '/Bancos_Dados/UFJF/Periodo_2019_1/SAD/'

In [6]:
#Bases

#base_Country = pd.read_csv(path_banco + 'Country.csv')
#base_League = pd.read_csv(path_banco + 'League.csv')
base_Match = pd.read_csv(path_banco + 'Match.csv',low_memory = False)
#base_Player = pd.read_csv(path_banco + 'Player.csv')
#base_Player_Attributes = pd.read_csv(path_banco + 'Player_Attributes.csv')
#base_Team= pd.read_csv(path_banco + 'Team.csv')
#base_Team_Attributes= pd.read_csv(path_banco + 'Team_Attributes.csv')
#base_sqlite_sequence= pd.read_csv(path_banco + 'sqlite_sequence.csv')


In [29]:
#Arrumando a base_Match
coluna = base_Match.loc[:,'possession']
coluna1 = base_Match.loc[pd.notnull(base_Match['possession'])]
coluna1 = coluna1.loc[:,['possession']]
coluna2 = coluna1.values

for linha in coluna:
    soup = BeautifulSoup(str(linha), 'html.parser')
    awaypos = soup.find_all('awaypos')
    homepos = soup.find_all('homepos')

    awaypos_num = []
    for pos in awaypos:
        awaypos_num.append(pos.get_text())
    homepos_num = []
    for pos in homepos:
        homepos_num.append(pos.get_text())
    print(awaypos_num)
    print(homepos_num)
    
#A ideia aqui eh criar duas colunas para possession, onde teremos possession_home e possession_away
        

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['44', '46', '46', '45']
['56', '54', '54', '55']
['35', '39', '35', '34']
['65', '61', '65', '66']
['55', '57', '52', '54']
['45', '43', '48', '46']
['50', '44', '59', '48']
['50', '56', '41', '52']
['49', '46', '51', '48']
['51', '54', '49', '52']
['54', '41', '49', '49']
['46', '59', '51', '51']
['51', '51', '47', '47']
['49', '49', '53', '53']
['42', '53', '49', '53']
['58', '47', '51', '47']
['51', '50', '47']
['49', '50', '53']
['43', '36', '35']
['57', '64', '65']
['41', '42', '41', '40']
['59', '58', '59', '60']
['37', '38']
['63', '62']
['52', '43', '51', '55']


['37', '33', '43', '41']
['63', '67', '57', '59']
['41', '42', '39', '42']
['59', '58', '61', '58']
['38', '27', '35', '50', '36', '36']
['62', '73', '65', '50', '64', '64']
['47', '46', '49', '48']
['53', '54', '51', '52']
['58', '62', '58', '58']
['42', '38', '42', '42']
['42', '46', '49', '47']
['58', '54', '51', '53']
['55', '48', '66', '51', '53']
['45', '52', '34', '49', '47']
['40', '33', '77', '47', '47', '46']
['60', '67', '23', '53', '53', '54']
['44', '45', '51', '54']
['56', '55', '49', '46']
['46', '48', '48', '53']
['54', '52', '52', '47']
['53', '49', '51', '46']
['47', '51', '49', '54']
['36', '46', '45', '48']
['64', '54', '55', '52']
['55', '51', '46', '45']
['45', '49', '54', '55']
['65', '52', '49', '46']
['35', '48', '51', '54']
['40', '55', '51', '50']
['60', '45', '49', '50']
['21', '22', '38', '39']
['79', '78', '62', '61']
['52', '39', '46', '45']
['48', '61', '54', '55']
['55', '57', '51', '44']
['45', '43', '49', '56']
['51', '51', '47', '48']
['49', '49', '5

['54', '58', '60', '59']
['46', '42', '40', '41']
['56', '43', '45', '54']
['44', '57', '55', '46']
['38', '44', '48', '50']
['62', '56', '52', '50']
['49', '50', '51', '47']
['51', '50', '49', '53']
['48', '28', '40', '39']
['52', '72', '60', '61']
['52', '51', '50', '50']
['48', '49', '50', '50']
['56', '45', '46', '48']
['44', '55', '54', '52']
['46', '47', '46', '47']
['54', '53', '54', '53']
['41', '40', '60', '51']
['59', '60', '40', '49']
['55', '43', '43', '45']
['45', '57', '57', '55']
['43', '53', '48', '51']
['57', '47', '52', '49']
['57', '49', '47', '52']
['43', '51', '53', '48']
['64', '57', '54', '59']
['36', '43', '46', '41']
['47', '47', '48', '54']
['53', '53', '52', '46']
['43', '48', '47', '45']
['57', '52', '53', '55']
['47', '45', '46', '47']
['53', '55', '54', '53']
['47', '45', '60', '55']
['53', '55', '40', '45']
['52', '47', '50', '52']
['48', '53', '50', '48']
['50', '52', '54', '57']
['50', '48', '46', '43']
['45', '50', '48', '48']
['55', '50', '52', '52']


['51', '54', '51', '52']
['41', '50', '50', '47']
['59', '50', '50', '53']
['49', '43', '47', '50']
['51', '57', '53', '50']
['44', '41', '43', '42']
['56', '59', '57', '58']
['54', '61', '59', '58']
['46', '39', '41', '42']
['53', '52', '45', '52']
['47', '48', '55', '48']
['48', '51', '45', '45']
['52', '49', '55', '55']
['43', '48', '49', '50']
['57', '52', '51', '50']
['41', '50', '48', '46']
['59', '50', '52', '54']
['61', '55', '56', '54']
['39', '45', '44', '46']
['46', '45', '40', '43']
['54', '55', '60', '57']
['47', '48', '52', '51']
['53', '52', '48', '49']
['48', '50', '51', '50']
['52', '50', '49', '50']
['35']
['65']
['44', '42', '39', '41', '42']
['56', '58', '61', '59', '58']
['54', '57', '55', '48']
['46', '43', '45', '52']
['37', '49', '57', '59']
['63', '51', '43', '41']
['38', '47', '50', '48']
['62', '53', '50', '52']
['39', '42', '42', '48', '48']
['61', '58', '58', '52', '52']
['48', '49', '48', '45']
['52', '51', '52', '55']
['49', '55', '50', '55']
['51', '45',

...

['38', '39', '38', '37']
[]
[]
['60', '66', '62', '59']
['40', '34', '38', '41']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['26', '30', '30', '32']
['74', '70', '70', '68']
[]
[]
['35', '37', '37', '36', '40', '42', '44']
['65', '63', '63', '64', '60', '58', '56']
['52', '51']
['48', '49']
['48', '53', '53', '51']
['52', '47', '47', '49']
[]
[]
['59', '60', '59', '57']
['41', '40', '41', '43']
['39', '41']
['61', '59']
['39', '38', '42', '42', '42', '41']
['61', '62', '58', '58', '58', '59']
[]
[]
['73', '71', '70', '73']
['27', '29', '30', '27']
['52', '59', '56', '55']
['48', '41', '44', '45']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['50', '47', '50', '52']
['50', '53', '50', '48']
['47', '46', '48', '46']
['53', '54', '52', '54']
['40', '46', '47', '44']
['60', '54', '53', '56']
['35', '39', '44', '43']
['65', '61', '56', '57']
['27', '28', '29', '27']
['73', '72', '71', '73']
[]
[]
[]
[]
['24', '29', '27', '28']
['76', '71', '73', '72']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['47', '53', '53', 

['35', '44', '47', '45']
['65', '56', '53', '55']
['36', '38', '37', '38']
['64', '62', '63', '62']
['49', '51', '52', '51']
['51', '49', '48', '49']
['59', '54', '55', '54']
['41', '46', '45', '46']
['30', '35', '41', '46']
['70', '65', '59', '54']
['72', '67', '66', '64']
['28', '33', '34', '36']
['49', '42', '43', '41']
['51', '58', '57', '59']
['59', '56', '52', '51']
['41', '44', '48', '49']
['42', '44', '45', '43']
['58', '56', '55', '57']
['52', '58', '59', '59']
['48', '42', '41', '41']
['52', '50', '50', '48']
['48', '50', '50', '52']
['30', '33', '31', '28']
['70', '67', '69', '72']
['43', '46', '49', '50']
['57', '54', '51', '50']
['51', '50', '46', '41']
['49', '50', '54', '59']
['60', '58', '54', '52']
['40', '42', '46', '48']
['32', '40', '47', '50']
['68', '60', '53', '50']
['69', '64', '61', '59']
['31', '36', '39', '41']
['57', '51', '55', '55']
['43', '49', '45', '45']
['36', '50', '52', '54']
['64', '50', '48', '46']
['56', '54', '57', '53']
['44', '46', '43', '47']


['45', '42', '45', '48']
['35', '33', '30', '30']
['65', '67', '70', '70']
['62', '59', '58', '57']
['38', '41', '42', '43']
['16', '23', '29', '29']
['84', '77', '71', '71']
['39', '40', '38', '43']
['61', '60', '62', '57']
['62', '58', '54']
['38', '42', '46']
['49', '57', '58', '59']
['51', '43', '42', '41']
['39', '45', '45', '44']
['61', '55', '55', '56']
['51', '50', '51', '45']
['49', '50', '49', '55']
['42', '43', '35', '35']
['58', '57', '65', '65']
['31', '34', '34', '34']
['69', '66', '66', '66']
['52', '43', '42', '43']
['48', '57', '58', '57']
['60', '61', '62']
['40', '39', '38']
['49', '42', '40', '45']
['51', '58', '60', '55']
['43', '46', '42']
['57', '54', '58']
['32', '41', '41', '38']
['68', '59', '59', '62']
['32', '32', '36', '41']
['68', '68', '64', '59']
['56', '52', '57', '59']
['44', '48', '43', '41']
['30', '28', '37', '38']
['70', '72', '63', '62']
['39', '40', '47', '50']
['61', '60', '53', '50']
['43', '52', '52', '55']
['57', '48', '48', '45']
['72', '72'

['54', '54', '45', '45', '54', '54', '52', '52']
['41', '41', '47', '47', '48', '48', '52', '52']
['59', '59', '53', '53', '52', '52', '48', '48']
['42', '42', '47', '47', '48', '48', '50', '50']
['58', '58', '53', '53', '52', '52', '50', '50']
['37', '37', '48', '48', '54', '54', '58', '58']
['63', '63', '52', '52', '46', '46', '42', '42']
['40', '40', '44', '44', '41', '41', '38', '38']
['60', '60', '56', '56', '59', '59', '62', '62']
['52', '52', '49', '49', '48', '48', '48', '48']
['48', '48', '51', '51', '52', '52', '52', '52']
['79', '79', '71', '71', '70', '70', '70', '70']
['21', '21', '29', '29', '30', '30', '30', '30']
['39', '39', '47', '47', '46', '46', '46', '46']
['61', '61', '53', '53', '54', '54', '54', '54']
['40', '40', '40', '40', '44', '44', '46', '46']
['60', '60', '60', '60', '56', '56', '54', '54']
['49', '49', '53', '53', '51', '51', '48', '48']
['51', '51', '47', '47', '49', '49', '52', '52']
['59', '59', '54', '54', '53', '53', '53', '53']
['41', '41', '46', '

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

In [30]:
base1 = base_Match.loc[:,['home_team_goal','away_team_goal','possession']]

base2 = base_Match.loc[:,['home_player_1','home_player_2','home_player_3','home_player_4','home_player_5',
                         'home_player_6','home_player_7','home_player_8','home_player_9','home_player_10','home_player_11',
                         'away_player_1','away_player_2','away_player_3','away_player_4','away_player_5','away_player_6',
                         'away_player_7','away_player_8','away_player_9','away_player_10','away_player_11', 'match_api_id',
                         'session','home_team_api_id','away_team_api_id','home_team_goal','away_team_goal']]

base3 = base_Match.loc[:,['home_team_goal', 'away_team_goal', 'shoton', 'shotoff']]

In [25]:
base1.mean()

home_team_goal    1.544594
away_team_goal    1.160938
dtype: float64

In [26]:
base2.mean()

home_player_1       7.663836e+04
home_player_2       1.068541e+05
home_player_3       9.160129e+04
home_player_4       9.454020e+04
home_player_5       1.095278e+05
home_player_6       1.023088e+05
home_player_7       9.728764e+04
home_player_8       1.072910e+05
home_player_9       1.111315e+05
home_player_10      1.056125e+05
home_player_11      1.034139e+05
away_player_1       7.662820e+04
away_player_2       1.076146e+05
away_player_3       9.112678e+04
away_player_4       9.508391e+04
away_player_5       1.098009e+05
away_player_6       1.023083e+05
away_player_7       9.789806e+04
away_player_8       1.092651e+05
away_player_9       1.110870e+05
away_player_10      1.071491e+05
away_player_11      1.049329e+05
match_api_id        1.195429e+06
session                      NaN
home_team_api_id    9.984372e+03
away_team_api_id    9.984475e+03
home_team_goal      1.544594e+00
away_team_goal      1.160938e+00
dtype: float64

In [27]:
base3.mean()

home_team_goal    1.544594
away_team_goal    1.160938
dtype: float64

In [33]:
base_Match.describe()

,id,country_id,league_id,stage,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
count,25979.000000,25979.000000,25979.000000,25979.000000,2.597900e+04,25979.000000,25979.000000,25979.000000,25979.000000,24158.000000,...,17097.000000,22568.000000,22568.000000,22568.000000,14162.000000,14162.000000,14162.000000,14161.000000,14161.000000,14161.000000
mean,12990.000000,11738.630317,11738.630317,18.242773,1.195429e+06,9984.371993,9984.475115,1.544594,1.160938,0.999586,...,4.622343,2.668107,3.899048,4.840281,2.498764,3.648189,4.353097,2.497894,3.660742,4.405663
std,7499.635658,7553.936759,7553.936759,10.407354,4.946279e+05,14087.453758,14087.445135,1.297158,1.142110,0.022284,...,3.632164,1.928753,1.248221,4.318338,1.489299,0.867440,3.010189,1.507793,0.868272,3.189814
min,1.000000,1.000000,1.000000,1.000000,4.831290e+05,1601.000000,1601.000000,0.000000,0.000000,0.000000,...,1.100000,1.030000,1.620000,1.080000,1.050000,1.450000,1.120000,1.040000,1.330000,1.120000
25%,6495.500000,4769.000000,4769.000000,9.000000,7.684365e+05,8475.000000,8475.000000,1.000000,0.000000,1.000000,...,2.500000,1.700000,3.300000,2.550000,1.670000,3.200000,2.500000,1.670000,3.250000,2.500000
50%,12990.000000,10257.000000,10257.000000,18.000000,1.147511e+06,8697.000000,8697.000000,1.000000,1.000000,1.000000,...,3.500000,2.150000,3.500000,3.500000,2.100000,3.300000,3.400000,2.100000,3.400000,3.400000
75%,19484.500000,17642.000000,17642.000000,27.000000,1.709852e+06,9925.000000,9925.000000,2.000000,2.000000,1.000000,...,5.250000,2.800000,4.000000,5.400000,2.650000,3.750000,5.000000,2.620000,3.750000,5.000000
max,25979.000000,24558.000000,24558.000000,38.000000,2.216672e+06,274581.000000,274581.000000,10.000000,9.000000,2.000000,...,41.000000,36.000000,26.000000,67.000000,21.000000,11.000000,34.000000,17.000000,13.000000,34.000000


In [34]:
base_Match.columns

Index(['id', 'country_id', 'league_id', 'season', 'stage', 'date',
       'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal',
       ...
       'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA'],
      dtype='object', length=115)

In [35]:
base_Match.shape

(25979, 115)

In [36]:
base_Match.count()

id                  25979
country_id          25979
league_id           25979
season              25979
stage               25979
date                25979
match_api_id        25979
home_team_api_id    25979
away_team_api_id    25979
home_team_goal      25979
away_team_goal      25979
home_player_X1      24158
home_player_X2      24158
home_player_X3      24147
home_player_X4      24147
home_player_X5      24147
home_player_X6      24147
home_player_X7      24147
home_player_X8      24147
home_player_X9      24147
home_player_X10     24147
home_player_X11     24147
away_player_X1      24147
away_player_X2      24147
away_player_X3      24147
away_player_X4      24147
away_player_X5      24147
away_player_X6      24147
away_player_X7      24147
away_player_X8      24147
                    ...  
B365H               22592
B365D               22592
B365A               22592
BWH                 22575
BWD                 22575
BWA                 22575
IWH                 22520
IWD         

In [37]:
base_Match.min()

id                                    1
country_id                            1
league_id                             1
season                        2008/2009
stage                                 1
date                2008-07-18 00:00:00
match_api_id                     483129
home_team_api_id                   1601
away_team_api_id                   1601
home_team_goal                        0
away_team_goal                        0
home_player_X1                        0
home_player_X2                        0
home_player_X3                        1
home_player_X4                        2
home_player_X5                        1
home_player_X6                        1
home_player_X7                        1
home_player_X8                        1
home_player_X9                        1
home_player_X10                       1
home_player_X11                       1
away_player_X1                        1
away_player_X2                        1
away_player_X3                        2


In [38]:
base_Match.max()

id                                25979
country_id                        24558
league_id                         24558
season                        2015/2016
stage                                38
date                2016-05-25 00:00:00
match_api_id                    2216672
home_team_api_id                 274581
away_team_api_id                 274581
home_team_goal                       10
away_team_goal                        9
home_player_X1                        2
home_player_X2                        8
home_player_X3                        8
home_player_X4                        8
home_player_X5                        9
home_player_X6                        9
home_player_X7                        9
home_player_X8                        9
home_player_X9                        9
home_player_X10                       9
home_player_X11                       7
away_player_X1                        6
away_player_X2                        8
away_player_X3                        9


In [39]:
base_Match.median()

id                    12990.00
country_id            10257.00
league_id             10257.00
stage                    18.00
match_api_id        1147511.00
home_team_api_id       8697.00
away_team_api_id       8697.00
home_team_goal            1.00
away_team_goal            1.00
home_player_X1            1.00
home_player_X2            2.00
home_player_X3            4.00
home_player_X4            6.00
home_player_X5            8.00
home_player_X6            3.00
home_player_X7            5.00
home_player_X8            6.00
home_player_X9            5.00
home_player_X10           5.00
home_player_X11           6.00
away_player_X1            1.00
away_player_X2            2.00
away_player_X3            4.00
away_player_X4            6.00
away_player_X5            8.00
away_player_X6            3.00
away_player_X7            5.00
away_player_X8            6.00
away_player_X9            5.00
away_player_X10           5.00
                       ...    
B365H                     2.10
B365D   